# Convolutional Neural Networks

## Load Data

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


2024-07-22 23:55:07.421414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 23:55:07.580420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 23:55:07.581060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 23:55:07.793552: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 23:55:10.026696: W tensorflow/compiler/tf

Num GPUs Available:  1


2024-07-22 23:55:12.474333: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 23:55:12.732455: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 23:55:12.745459: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
images_train = np.expand_dims(np.load("../data/images/images_train.npy") / 255.0, -1)
images_val = np.expand_dims(np.load("../data/images/images_val.npy") / 255.0, -1)
images_test = np.expand_dims(np.load("../data/images/images_test.npy") / 255.0, -1)
num_samples_train = images_train.shape[0]
num_samples_val = images_val.shape[0]
num_samples_test = images_test.shape[0]

hog_train = np.load("../data/hog/hog_train.npy").reshape(num_samples_train, -1) * 255 / 255.0
hog_val = np.load("../data/hog/hog_val.npy").reshape(num_samples_val, -1) * 255 / 255.0
hog_test = np.load("../data/hog/hog_test.npy").reshape(num_samples_test, -1) * 255 / 255.0

y_train = np.load("../data/images/label_ids_train.npy")
y_val = np.load("../data/images/label_ids_val.npy")
y_test = np.load("../data/images/label_ids_test.npy")


In [3]:
labels_train = np.load("../data/images/labels_train.npy")
class_lookup = list(set(zip(y_train, labels_train)))
class_lookup = {id_value: class_value for id_value, class_value in class_lookup}
class_lookup


{1: 'spiral',
 0: 'hairline',
 5: 'pathological',
 4: 'dislocation',
 7: 'oblique',
 2: 'greenstick',
 9: 'avulsion',
 8: 'impacted',
 3: 'comminuted',
 6: 'longitudinal'}

In [4]:
hog_train.shape

(890, 262144)

In [5]:
image_input = layers.Input(shape=(images_train.shape[1], images_train.shape[2], images_train.shape[3]))
x = layers.Conv2D(16, (3, 3), activation='relu')(image_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)

hog_input = layers.Input(shape=(hog_train.shape[1],))
x = layers.Concatenate()([x, hog_input])

x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=[image_input, hog_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


2024-07-22 23:55:14.563708: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 23:55:14.585134: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 23:55:14.600053: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 510, 510,  │        160 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 255, 255,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 253, 253,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 126, 126,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 124, 124,  │      9,248 │ max_pooling2d_1[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 492032)    │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 262144)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 754176)    │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │ 48,267,328 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10)        │        650 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 48,282,026 (184.18 MB)

 Trainable params: 48,282,026 (184.18 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.fit([images_train, hog_train], y_train, validation_data=([images_val, hog_val], y_val), epochs=10, batch_size=32)


Epoch 1/10


I0000 00:00:1721706921.875649  631781 service.cc:145] XLA service 0x7594b8006c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721706921.876289  631781 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-22 23:55:22.011698: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-22 23:55:22.383775: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


 3/28 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.0955 - loss: 3.2319 

I0000 00:00:1721706932.205804  631781 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


28/28 ━━━━━━━━━━━━━━━━━━━━ 23s 379ms/step - accuracy: 0.1378 - loss: 3.0370 - val_accuracy: 0.1717 - val_loss: 2.2230
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.5392 - loss: 1.4062 - val_accuracy: 0.1919 - val_loss: 2.4538
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.8447 - loss: 0.5886 - val_accuracy: 0.2828 - val_loss: 2.5920
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9805 - loss: 0.1712 - val_accuracy: 0.2626 - val_loss: 2.7516
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.9959 - loss: 0.0594 - val_accuracy: 0.2828 - val_loss: 2.9972
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9916 - loss: 0.0336 - val_accuracy: 0.2929 - val_loss: 3.1660
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.9969 - loss: 0.0263 - val_accuracy: 0.3434 - val_loss: 3.1438
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.9967 - loss: 0.0124 - val_accuracy: 0.3131 - val_loss: 

In [7]:
y_pred = model.predict([images_test, hog_test])
y_pred = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, output_dict=True)
report = pd.DataFrame(report).transpose()
report.index = report.index.map(lambda x: class_lookup[int(x)] if x.isdigit() else x)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
report


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 286ms/step
Accuracy: 0.2857142857142857
Classification Report:


,precision,recall,f1-score,support
hairline,0.142857,0.100000,0.117647,10.000000
spiral,0.333333,0.083333,0.133333,12.000000
greenstick,0.416667,0.312500,0.357143,16.000000
comminuted,0.227273,0.357143,0.277778,14.000000
dislocation,0.333333,0.631579,0.436364,19.000000
pathological,0.222222,0.222222,0.222222,18.000000
longitudinal,0.400000,0.166667,0.235294,12.000000
oblique,0.454545,0.312500,0.370370,16.000000
impacted,0.250000,0.222222,0.235294,9.000000
avulsion,0.166667,0.214286,0.187500,14.000000
